In [3]:
// Deno imports
import { Stagehand } from "npm:@browserbasehq/stagehand";
import { z } from "npm:zod";
import yaml from "npm:js-yaml";

const cdpPort = 9222;
let stagehand = null;



In [4]:
//Initialize Stagehand
(async () => {
  stagehand = new Stagehand({
    verbose: 1, // Verbosity level for logging: 0 = silent, 1 = info, 2 = all
    domSettleTimeoutMs: 30_000, // Timeout for DOM to settle in milliseconds

    // LLM configuration
    modelName: "gpt-4o", // Name of the model to use
    modelClientOptions: {
      apiKey: process.env.OPENAI_API_KEY,
    }, // Configuration options for the model client

    // Browser configuration
    env: "LOCAL", // Environment to run in: LOCAL or BROWSERBASE
    localBrowserLaunchOptions: {
      cdpUrl: `http://localhost:${cdpPort}`,
      viewport: {
        width: 1024,
        height: 768,
      },
    }, // Configuration options for the local browser
  });
  await stagehand.init();

  console.log("Stagehand initialized successfully!");
})();


Promise { <pending> }

Stagehand initialized successfully!


In [8]:
(async () => {
    await stagehand.page.goto('https://google.com')
})();

Promise { <pending> }